# !!! Non-operational !!!

### This tutorial is currently non-operational. When non-legacy code is operational, these tutorials are to be adapted.

I've split the Notebook in two sections. Thompson Sampling and Results Analysis. 

The files provided for the input are accessible with:
```
../examples/docking_scores/{file_name}
../examples/input_files/{file_name}
```

output files should always be placed in (this ensures any file created is not uploaded to the GitHub repo accidently):
```
./tmp
```

I've tried going through the notebook using the data provided, but encountered issues not all files seem to be available. 

I've added comments in *italics*

# Assessment of the Results of Docking for Thrombin

In [1]:
import sys
import os
from collections import Counter
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display
import polars as pl
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# Add the root directory of your project to the Python path
project_root = './tmp'
if project_root not in sys.path:
    sys.path.append(project_root)
# Import functionalities from enumeration_utils.py
from TACTICS.library_enumeration.enumeration_utils import find_reactants_from_product_code
# Import functionalities from library_analysis_utils.py
from TACTICS.library_analysis.library_analysis_utils import compile_product_scores, compile_product_smiles

/home/remco/.conda/envs/tactics/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/remco/apps/TACTICS/src/TACTICS/thompson_sampling/core/evaluators.py:16: UserWarning: Openeye packages not available in this environment; do not attempt to use ROCSEvaluator or FredEvaluator
  warnings.warn(f"Openeye packages not available in this environment; do not attempt to use ROCSEvaluator or "


In [2]:
def read_smi_file(file_path):
    """
    Read a .smi file and create a dictionary with building block codes as keys and SMILES as items.
    
    :param file_path: The path to the .smi file.
    :return: A dictionary with building block codes as keys and SMILES as items.
    """
    building_block_dict = {}
    
    with open(file_path, 'r') as file:
        for line in file:
            smiles, code = line.strip().split()
            building_block_dict[code] = smiles
    
    return building_block_dict

## Compile the scores and dictionary of product codes

*No Linear_amide folder present*

In [3]:
# Generate Dictionary to map product codes to product SMILES
#prod_SMILES_dir = "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide" -> to be placed in:
prod_SMILES_dir = "../examples/Linear_amide"
prod_smiles_dict = compile_product_smiles(prod_SMILES_dir)

FileNotFoundError: [Errno 2] No such file or directory: '../examples/Linear_amide'

*_______ Unable to continue further _______*

In [ ]:
# Import Data from Brute force docking
prod_scores_dir = "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide/docking_scores"
prod_scores_df = compile_product_scores(prod_scores_dir)

In [ ]:
# Generate a Dictionary to map product codes to product SMILES (protected products)
prod_SMILES_dir_prot = "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide/protected_prods"
prod_smiles_dict_prot = compile_product_smiles(prod_SMILES_dir_prot)

prod_scores_dir = "/Users/aakankschitnandkeolyar/Desktop/TS_Chem_Space/Thrombin/Linear_amide/docking_scores"
prod_scores_df = compile_product_scores(prod_scores_dir)

In [ ]:
# Add a new column for SMILES strings by mapping Product_Code to SMILES using the dictionary
prod_scores_df = prod_scores_df.with_columns(
    pl.Series("SMILES", [prod_smiles_dict_prot.get(code, None) for code in prod_scores_df["Product_Code"].to_list()])
)

# Select only the SMILES strings and scores
smiles_scores_df = prod_scores_df.select(["SMILES", "Scores"])

# Write the SMILES strings and scores to a .csv file
output_file = os.path.join(prod_scores_dir, "smiles_scores.csv")
smiles_scores_df.write_csv(output_file)

print(f"SMILES strings and scores have been written to {output_file}")

## Docking Success Rate
Here I want to estimate what percentage of the library was successfully docked. Additionally, I would like to see if there was any common substructure amongst the compounds that failed to dock.

In [ ]:
# Find the products that were not docked 
undocked_prods = set(prod_smiles_dict.keys()) - set(prod_scores_df["Product_Code"])
print(f"Number of undocked products: {len(undocked_prods)}")
print("Undocked products:")
for prod in undocked_prods:
    print(prod)

In [ ]:
import polars as pl

# Create a new DataFrame for undocked products with a score of +10
undocked_df = pl.DataFrame({
    "Product_Code": list(undocked_prods),
    "Scores": [10.0] * len(undocked_prods)  # Assign a score of +10 to all undocked products
})

# Concatenate the undocked products DataFrame with the existing prod_scores_df
prod_scores_df = pl.concat([prod_scores_df, undocked_df])

# Print the updated DataFrame
print(f"Updated prod_scores_df with undocked products:")
print(prod_scores_df)

In [ ]:
# Define the output file path
output_file = os.path.join(prod_scores_dir, "product_scores.csv")

# Write the Polars DataFrame to a .csv file
prod_scores_df.write_csv(output_file)

print(f"Product scores have been written to {output_file}")

In [ ]:
# Docking success rate
# Total number of products in the library
total_products = len(prod_smiles_dict)

# Number of undocked products
num_undocked = len(undocked_prods)

# Number of successfully docked products
num_docked = total_products - num_undocked

# Calculate percentages
percent_docked = (num_docked / total_products) * 100
percent_undocked = (num_undocked / total_products) * 100

# Print the results
print(f"Percentage of compounds successfully docked: {percent_docked:.2f}%")
print(f"Percentage of compounds that did not dock: {percent_undocked:.2f}%")

Docking was successful, given nearly all of the compounds docked. Lets check to see if there were any building blocks that were common amongst the ones that failed.

In [ ]:
from collections import Counter

# Count occurrences of each building block in the undocked product codes
building_block_counts = Counter()
for prod in undocked_prods:
    building_blocks = prod.split("_")  # Split the product code by "_"
    building_block_counts.update(building_blocks)

# Print the occurrences neatly as a table
print(f"{'Building Block':<20}{'Occurrences':<10}")
print("-" * 30)
for block, count in building_block_counts.items():
    print(f"{block:<20}{count:<10}")

## Lets do some analysis on the hits
I am screening for the top 1% of ligands. The ChemGauss score that `openeye` utilizes returns a more negative value for a ligand with greater binding affinity.

In [ ]:
# Lets find the top 20 most commonly occurring products

total_molecules = 5000  # The top 1% of products

sorted_prod_scores_df = prod_scores_df.sort("Scores", descending=False)
top_products_df = sorted_prod_scores_df.head(200)

In [ ]:
# For each of the products extract the building blocks
# This is to count the occurrences of each building block in the top 1% of products
top_building_blocks = []
for code in top_products_df["Product_Code"]:
    building_blocks = code.split('_')
    top_building_blocks.extend(building_blocks)

In [ ]:
# Count the occurrences of each building block
building_block_counts = Counter(top_building_blocks)

# Find the top 20 most commonly occurring building blocks
top_20_common_building_blocks = building_block_counts.most_common(20)

# Print the table header
print(f"{'Building Block':<20}{'Frequency':<10}")
print("-" * 30)

# Print the building blocks and their frequencies
for block, count in top_20_common_building_blocks:
    print(f"{block:<20}{count:<10}")

### Visualize the Building Blocks 
 That were enriched in the top 20 building blocks 

In [ ]:
amino_acid_bb_dict = read_smi_file('/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/amino_acids_deprotected.smi')
acids_bb_dict = read_smi_file('/Users/aakankschitnandkeolyar/Desktop/PRISMS/tests/Data/Thrombin/input_files/acids.smi')

In [ ]:
# Get the SMILES for the top 20 building blocks
top_20_smiles = []
labels=[]
for block, _ in top_20_common_building_blocks:
    if 'AA' in block:
        top_20_smiles.append(amino_acid_bb_dict[block])
    elif 'CA' in block:
        top_20_smiles.append(acids_bb_dict[block])
    labels.append(block)

In [ ]:
# Create RDKit molecule objects
molecules = [Chem.MolFromSmiles(smiles) for smiles in top_20_smiles]

In [ ]:
# Visualize the molecules in a grid (5 molecules per row)
Draw.MolsToGridImage(molecules, molsPerRow=5, subImgSize=(300, 300), legends=labels)

### Now look at the top 20 building blocks in each position
This will help in understanding the enrichment of building blocks in certain positions.

In [ ]:
# Extract the product codes as a list
product_codes = top_products_df["Product_Code"].to_list()

# Initialize counters for each position
position_counters = []

# Iterate through the product codes
for product_code in product_codes:
    building_blocks = product_code.split("_")  # Split the product code by "_"
    # Ensure the position_counters list is large enough to handle all positions
    while len(position_counters) < len(building_blocks):
        position_counters.append(Counter())
    # Update the counters for each position
    for i, block in enumerate(building_blocks):
        position_counters[i][block] += 1

# Find the top 20 building blocks for each position
for i, counter in enumerate(position_counters):
    print(f"Top 20 building blocks for position {i + 1}:")
    print(f"{'Building Block':<20}{'Frequency':<10}")
    print("-" * 30)
    for block, count in counter.most_common(20):
        print(f"{block:<20}{count:<10}")
    print("\n")

In [ ]:
# Collect the top 20 building blocks for each position
top_20_building_blocks_per_position = []
for counter in position_counters:
    top_20_blocks = [block for block, _ in counter.most_common(20)]
    top_20_building_blocks_per_position.append(top_20_blocks)

# Convert the list to a tuple
top_20_building_blocks_tuple = tuple(top_20_building_blocks_per_position)

In [ ]:
# Combine the two dictionaries
combined_smiles_dict = {**amino_acid_bb_dict, **acids_bb_dict}

# Iterate through each position's top 20 building blocks
for i, counter in enumerate(position_counters):
    # Get the top 20 building blocks for the current position
    top_20_blocks = counter.most_common(20)
    
    # Create a list of RDKit molecules and their labels
    mols = []
    legends = []
    for block, freq in top_20_blocks:
        if block in combined_smiles_dict:  # Use the combined dictionary
            smiles = combined_smiles_dict[block]
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                mols.append(mol)
                # Add building block name and frequency on the first line, fraction on the second line
                legends.append(f"{block} (Freq: {freq})\nFraction: {round((freq / total_molecules) * 100, 2)}%")
    
    # Visualize the molecules in a grid
    img = Draw.MolsToGridImage(
        mols, legends=legends, molsPerRow=5, subImgSize=(300, 300)
    )
    
    # Display the title and the image
    print(f"Top 20 Building Blocks for Position {i + 1}")
    display(img)  # Display the image in the Jupyter Notebook

In [ ]:
def get_top_building_blocks(top_products_df, top_n, Maximize=False):
    """
    Extract the top 20 building blocks for each position from the top N products.
    
    :param Maximize: Sort the building blocks in descending order. If larger values are better scores.
    :param top_products_df: Polars DataFrame containing product codes.
    :param top_n: Number of top products to consider.
    :return: A tuple of lists containing the top 20 building blocks for each position.
    """
    # Get the top N products
    if isinstance(top_products_df, pd.DataFrame):
        top_products_df = top_products_df.sort_values(by="Scores", ascending=Maximize)
    elif isinstance(top_products_df, pl.DataFrame):
        top_products_df = top_products_df.sort("Scores", descending=Maximize)
    else:
        raise TypeError("Input must be either a pandas DataFrame or a polars DataFrame")

    top_n_df = top_products_df.head(top_n)
    
    # Extract the product codes as a list
    product_codes = top_n_df["Product_Code"].to_list()

    # Initialize counters for each position
    position_counters = []

    # Iterate through the product codes
    for product_code in product_codes:
        building_blocks = product_code.split("_")  # Split the product code by "_"
        # Ensure the position_counters list is large enough to handle all positions
        while len(position_counters) < len(building_blocks):
            position_counters.append(Counter())
        # Update the counters for each position
        for i, block in enumerate(building_blocks):
            position_counters[i][block] += 1

    # Collect the top 20 building blocks for each position
    top_20_building_blocks = []
    for counter in position_counters:
        top_20_building_blocks.append([block for block, _ in counter.most_common(20)])
    
    return tuple(top_20_building_blocks)

In [ ]:
# Function to check overlap for each position and print the total overlap
def check_overlap(top_20_smaller, top_20_5000):
    overlap_results = []
    total_overlap = 0  # To track the total number of overlapping compounds across all positions

    for position, smaller_list in enumerate(top_20_smaller):
        # Get the corresponding list from the top 5000
        larger_list = top_20_5000[position]
        # Find the intersection
        overlap = set(smaller_list).intersection(set(larger_list))
        overlap_results.append((position + 1, len(overlap), overlap))  # Position is 1-based
        total_overlap += len(overlap)  # Add the count of overlaps for this position

        # Print the overlap for the current position
        print(f"Position {position + 1}: {len(overlap)} building blocks overlap")

    # Print the total overlap
    print(f"Total number of overlapping compounds across all positions: {total_overlap}")
    return overlap_results

In [ ]:
def visualize_overlapping_blocks(overlap, combined_smiles_dict):
    """
    Visualize the overlapping building blocks for each position.
    
    :param overlap: Overlap data for a specific cutoff (list of tuples with position, count, and blocks).
    :param combined_smiles_dict: Dictionary mapping building block codes to SMILES strings.
    """
    for position, count, blocks in overlap:
        mols = []
        legends = []
        for block in blocks:
            if block in combined_smiles_dict:
                smiles = combined_smiles_dict[block]
                mol = Chem.MolFromSmiles(smiles)
                if mol:
                    mols.append(mol)
                    # Add only the building block name to the legend
                    legends.append(f"{block}")
        
        # Visualize the molecules in a grid
        img = Draw.MolsToGridImage(mols, legends=legends, molsPerRow=5, subImgSize=(300, 300))
        
        # Display the title and the image
        print(f"Overlapping Building Blocks for Position {position}")
        display(img)

In [ ]:
# Example usage
top_200_building_blocks = get_top_building_blocks(prod_scores_df, 200)
top_400_building_blocks = get_top_building_blocks(prod_scores_df, 400)
top_800_building_blocks = get_top_building_blocks(prod_scores_df, 800)
top_1000_building_blocks = get_top_building_blocks(prod_scores_df, 1000)
top_5000_building_blocks = get_top_building_blocks(prod_scores_df, 5000)

# Print results for top 200 as an example
for i, blocks in enumerate(top_200_building_blocks):
    print(f"Top 20 building blocks for position {i + 1} (Top 200 products):")
    print(blocks)
    print("\n")

In [ ]:
# Check overlaps for each cutoff
overlap_200 = check_overlap(top_200_building_blocks, top_5000_building_blocks)
overlap_400 = check_overlap(top_400_building_blocks, top_5000_building_blocks)
overlap_800 = check_overlap(top_800_building_blocks, top_5000_building_blocks)
overlap_1000 = check_overlap(top_1000_building_blocks, top_5000_building_blocks)

# Print results
for cutoff, overlap in zip([200, 400, 800, 1000], [overlap_200, overlap_400, overlap_800, overlap_1000]):
    print(f"Comparing enriched building blocks from the top {cutoff} compounds with the top 5000 compounds:")
    for position, count, blocks in overlap:
        print(f"  Position {position}: {count} building blocks overlap")
        print(f"    Overlapping building blocks: {', '.join(blocks)}")
    print("\n")

In [ ]:
# Example usage
combined_smiles_dict = {**amino_acid_bb_dict, **acids_bb_dict}  # Combine dictionaries for SMILES lookup
total_molecules_200 = 200
total_molecules_400 = 400
total_molecules_800 = 800
total_molecules_1000 = 1000

# Visualize overlaps for each cutoff
print("Visualizing overlaps for top 200 compounds:")
visualize_overlapping_blocks(overlap_200, combined_smiles_dict)

print("Visualizing overlaps for top 400 compounds:")
visualize_overlapping_blocks(overlap_400, combined_smiles_dict)

print("Visualizing overlaps for top 800 compounds:")
visualize_overlapping_blocks(overlap_800, combined_smiles_dict)

print("Visualizing overlaps for top 1000 compounds:")
visualize_overlapping_blocks(overlap_1000, combined_smiles_dict)